<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_training%2C_Entity_Ruler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This example demonstrates how to use rules-augmented model-based NER to locate stock codes

In [1]:
!pip install --quiet -U spacy
!python -m spacy download zh_core_web_lg
!git clone http://github.com/raymondwcs/learning_spacy

     |████████████████████████████████| 5.9 MB 4.5 MB/s 
     |████████████████████████████████| 456 kB 54.5 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 28.1 MB/s 
     |████████████████████████████████| 623 kB 69.5 MB/s 
     |████████████████████████████████| 603.8 MB 8.3 kB/s 
     |████████████████████████████████| 2.4 MB 4.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')
Cloning into 'learning_spacy'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 237 (delta 135), reused 57 (delta 19), pack-reused 0
Receiving objects: 100% (237/237), 66.24 MiB | 28.01 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [2]:
import spacy
import random
from spacy.training.example import Example
from spacy import displacy
import re


In [3]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")

# Part 1 - Train model-based NER

## Data for training the model-based NER
Training can also be done via CLI.  Details below.

https://github.com/raymondwcs/learning_spacy/tree/main/NER_Training_CLI

In [4]:
TRAIN_DATA = [
    ("一齊係國企成份股調整期間 大家一齊不問價掃貨 尤其是386 857 成份股佔指數比重只係5%左右 冇咩受新計法影響 挾死班歐美鬼佬 港股10月29000 遠必挾之", 
        {"entities": [(26,29,"STOCK"),(30,33,"STOCK")]}),
    ("高盛：維持對騰訊(0700)買入評級 目標價705港元高盛發表報告指,與騰訊管理層於路演活動溝通後,重申對騰訊嘅積極正面睇法。",
        {"entities": [(6,8,"STOCK")]}),
    ("中電、匯控、恒大8月暴升8.5%。",
        {'entities': [(0,2,"STOCK"),(3,5,"STOCK"),(6,8,"STOCK")]}),
    ("匯控(00005)將於下周一（2日）公布2021年度中期業績。",
        {'entities': [(0,2,"STOCK")]}),
    ("中電，匯控，港交所齊齊跌8.5%金融海嘯後最差。",
        {"entities": [(0,2,"STOCK"),(3,5,"STOCK"),(6,9,"STOCK")]}),    
    ("港燈將恢復派息",
        {"entities": [(0,2,"STOCK")]})
]

In [5]:
ner=nlp.get_pipe("ner")

# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

## Start training

In [6]:
epoch = 40
optimizer = nlp.resume_training()

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA,size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.2,sgd=optimizer,losses=losses)
    print("Losses", losses)

Losses {'ner': 29.80569051964155}
Losses {'ner': 21.33120996992286}
Losses {'ner': 17.264909458008084}
Losses {'ner': 17.34275606229609}
Losses {'ner': 13.694723675889215}
Losses {'ner': 12.269792577459475}
Losses {'ner': 10.499901448728323}
Losses {'ner': 7.881121755061969}
Losses {'ner': 4.01600178885937}
Losses {'ner': 4.973818328274774}
Losses {'ner': 1.0109716565409617}
Losses {'ner': 0.3056274061318054}
Losses {'ner': 0.09899645444205335}
Losses {'ner': 0.22101525751172638}
Losses {'ner': 0.006302541398891955}
Losses {'ner': 0.037598553872680594}
Losses {'ner': 1.1605761572543476}
Losses {'ner': 0.7773343336417698}
Losses {'ner': 1.198540101539233e-06}
Losses {'ner': 5.229506106318361e-07}
Losses {'ner': 0.09386078555957031}
Losses {'ner': 1.3956495924558543e-07}
Losses {'ner': 5.3155734148987865e-06}
Losses {'ner': 3.2438042871598963e-06}
Losses {'ner': 2.222690839707944e-05}
Losses {'ner': 2.6733277308265385e-07}
Losses {'ner': 8.201469372117117e-08}
Losses {'ner': 2.5125684890

## Debug

In [7]:
for text, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    # print(text,ent)
    # print(text[ent[0]:ent[1]])
    doc = nlp(text)
    char_span = doc.char_span(ent[0],ent[1])
    if char_span is None:  # start and end don't map to tokens
        print("Misaligned tokens", text, ent)

## Save trained model to disk

In [8]:
nlp.to_disk('./ner_model')

# Part 2

## Load trained model from disk

In [20]:
nlp = spacy.load('./ner_model')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")

## Define custom rules for EntityRuler

In [21]:
!gdown --id 1eo5O6h69_cK09oeYoy9nHp3X8OJ0hKuF

Downloading...
From: https://drive.google.com/uc?id=1eo5O6h69_cK09oeYoy9nHp3X8OJ0hKuF
To: /content/stockcodes.tsv
100% 55.7k/55.7k [00:00<00:00, 38.9MB/s]


In [22]:
from pandas import *

# https://docs.google.com/spreadsheets/d/1-btKGkOo_ywPeH8qTlJL0IKF1bxeaPEKTVhAOPeYAU8/edit?usp=sharing
data = read_csv("stockcodes.tsv",delimiter="\t",dtype=str) 

stockcodes = data.iloc[:,0].to_list()   # STOCK
orgs = data.iloc[:,1].to_list()         # ORG-S

org_alias = ['5號仔','大笨象']           # ORG-S-A

print(stockcodes[:10])
print(orgs[:10])
print(org_alias[:10])

['0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011']
['中電控股', '中華煤氣', '九龍倉集團', '滙豐控股', '電能實業', '凱富能源', '電訊盈科', '九號通運', '恒隆集團', '恒生銀行']
['5號仔', '大笨象']


In [23]:
patterns = [
    {"label": "ORG-S",  "pattern": [{'ORTH': {'IN': orgs}}]},
    {"label": "ORG-S-A","pattern": [{'ORTH': {'IN': org_alias}}]},
    {"label": "STOCK",  "pattern": [{"IS_DIGIT": True},{"ORTH":"."},{"ORTH": "HK"}]},
    {"label": "STOCK",  "pattern": [{"TEXT": {"REGEX":"\d+\.HK"}}]},
    {"label": "STOCK",  "pattern": [{"ORTH": "("},{"IS_DIGIT": True},{"ORTH": ")"}]},
    {"label": "STOCK2", "pattern": [{"IS_DIGIT": True},{"ENT_TYPE": "ORG-S"}]},
    {"label": "STOCK2", "pattern": [{"POS": "NUM"},{"POS": "NOUN"}]},
    {"label": "STOCK2", "pattern": [{"POS": "NUM"},{"POS": "PROPN"}]},
    {"label": "STOCK3", "pattern": [{'TEXT': {'IN': stockcodes}}]},
]

if "entity_ruler" in nlp.pipe_names:
  nlp.remove_pipe("entity_ruler")

entity_ruler = nlp.add_pipe("entity_ruler", before='ner')
entity_ruler.add_patterns(patterns)

## Check stock suffix ('STOCK', 'STOCK3')

In [24]:
from spacy.language import Language
from spacy.tokens import Span

def print_ent_details(ents):
    ent_details = []
    for ent in ents:
      ent_details.append([ent.text, ent.label_])
    print('doc.ents: {}'.format(ent_details))

@Language.component("check_stockno_suffix")
def remove_stockno_suffix(doc):
    # print_ent_details(doc.ents)
    new_ents = []
    for i in range(len(doc.ents)):
      if doc.ents[i].label_ == "STOCK3" or doc.ents[i].label_ == "STOCK":
        token      = doc.ents[i]
        next_token = doc[doc.ents[i].start+1]
        # print('[{},{},{}]'.format(doc.ents[i].text,doc.ents[i].label_,next_token.text))
        # print(token.text,next_token.text)
        if  not re.search(r'[一二三四十五六七八九十零百千萬億壹貳參叄肆伍陸柒捌玖拾佰仟]+',token.text) and \
            not re.search(r'[蚊號]',token.text) and \
            not re.search(r'[元文蚊]',next_token.text):
          new_ent = Span(doc, doc.ents[i].start, doc.ents[i].end, label="STOCK")
          new_ents.append(new_ent)
      else:
        new_ents.append(doc.ents[i])
    doc.ents = new_ents
    # print_ent_details(doc.ents)
    return doc

# Add the component after the named entity recognizer
if "check_stockno_suffix" in nlp.pipe_names:
  nlp.remove_pipe("check_stockno_suffix")

nlp.add_pipe("check_stockno_suffix", after="ner")

<function __main__.remove_stockno_suffix>

## Remove stock noun suffix ('STOCK2')

In [25]:
from spacy.language import Language
from spacy.tokens import Span

@Language.component("remove_stockno_suffix")
def remove_stockno_suffix(doc):
    new_ents = []
    for ent in doc.ents:
        if ent.label_ == "STOCK2":  #and ent.start != 0:
            next_token = doc[ent.start + 1]
            if next_token.pos_ == "NOUN" or next_token.pos_ == "PROPN":
                new_ent = Span(doc, ent.start, ent.end - 1, label="STOCK")
                new_ents.append(new_ent)
        else:
            new_ents.append(ent)
    doc.ents = new_ents
    return doc

# Add the component after the named entity recognizer
if "remove_stockno_suffix" in nlp.pipe_names:
  nlp.remove_pipe("remove_stockno_suffix")

nlp.add_pipe("remove_stockno_suffix", after="ner")

<function __main__.remove_stockno_suffix>

# Test the trained NER model and EntityRuler

In [26]:
sentences = [
  "睇好港鐡(0066.HK)強烈建議買入 😎",
  "內地疫情受控,旅遊相關股同程藝龍(780)最近區間上落橫行,大戶收集似近完成,中線支持位$13.83,可考慮作中長線投資。",
  "睿見教育（6068.HK）：凈利與現金流大增超四成，估值吸引力凸顯，獲大行睇多	隻1765都賺多幾成 升左少少就跌凸 支股即係支股  🤷 ",
  "密切留意鋼鐵股，581中國東方，323馬鋼，347鞍鋼，1053重鋼，2600中鋁！",
  '入左中女 (308)  升四成，放定加馬	all in pk 左,起唔翻身  加一注 算, 3元走人!  贏咗購買 tesla 電車  輸咗就pk',
  '各位愛國人仕請支持883號中海油	有你地支持 相信中海油股價今日可以超英趕美  節節上升 多謝各位支持  883 號中海油 油中矛台 😎 ',
  '1727 終於止跌。回升中。	正常今日會反彈既  問題係彈完會企得穩收市嗎?  見佢買貨多過出貨  但都係俾人㩒住升5上去',
  '5號仔大笨象有冇機會企番起身	放心,我信佢會升上去 Ps: 33文入咗貨',
  '981中芯國際聽日爆升！淨利潤超預期，第二季度淨利潤1.38億美元，市場預期盈利9519萬美元，舊年同期盈利1854萬美元，同比增長6.44倍	35應該無問題 今月上返40蚊 GOGO'
]

# def replace_zh_punctuation(sentence):
#   str = sentence
#   str = str.replace("（", "(")
#   str = str.replace("）", ")")
#   return(str)

for sentence in sentences:
  # sentence = replace_zh_punctuation(sentence)
  doc = nlp(sentence)
  displacy.render(doc,style='ent',jupyter=True)
  print()